In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
warnings.filterwarnings('ignore')

In [4]:
#充电需求

In [5]:
station_info = pd.read_csv(r'D:\北大深\华为充电桩网络优化\第二阶段城市场景\仿真结果\output_10W_v2\output_10W_v2\station_infos.csv')
station_info = station_info[station_info['num_current_car']>0]
# 仿真步长
step_length = 5*60

import ast
station_info_table = station_info[['station_id','lon','lat','max_capacity','charge_speed_station']].drop_duplicates().copy()
station_info = station_info[['station_id','time','current_car','waiting_car']]

station_info['time'] = pd.to_datetime(station_info['time'])
station_info['current_car'] = station_info['current_car'].apply(lambda a:ast.literal_eval(a))
station_info['waiting_car'] = station_info['waiting_car'].apply(lambda a:ast.literal_eval(a))
station_info.sort_values(by=['station_id','time'],inplace=True)


# 将current_car和waiting_car列转换为车辆记录的列表
current_car_infos = station_info[['station_id','time','current_car']].explode('current_car')
current_car_infos = current_car_infos[~current_car_infos['current_car'].isnull()]
current_car_infos = current_car_infos.sort_values(by=['current_car','time'])[['current_car','time','station_id']]
waiting_car_infos = station_info[['station_id','time','waiting_car']].explode('waiting_car')
waiting_car_infos = waiting_car_infos[~waiting_car_infos['waiting_car'].isnull()]
waiting_car_infos = waiting_car_infos.sort_values(by=['waiting_car','time'])[['waiting_car','time','station_id']]


def get_charging_order(current_car_infos):
    current_car_infos['timegap'] = current_car_infos['time'].diff().dt.total_seconds().fillna(1000000).astype(int)
    current_car_infos['order_id'] = (current_car_infos['timegap']>step_length).cumsum()
    charge_info_s = current_car_infos.groupby(['current_car','order_id']).first().reset_index()
    charge_info_e = current_car_infos.groupby(['current_car','order_id']).last().reset_index()
    charging_order = pd.merge(charge_info_s,charge_info_e,on=['current_car','order_id','station_id'])
    charging_order
    charging_order = charging_order[['current_car','order_id','time_x','time_y','station_id']]
    charging_order.columns = ['carid','order_id','stime','etime','station_id']
    charging_order['duration'] = (charging_order['etime']-charging_order['stime']).dt.total_seconds()
    charging_order = charging_order[charging_order['duration']>0]
    return charging_order
# 计算充电信息
charging_orders = get_charging_order(current_car_infos)
# 计算等待信息
waiting_orders =  get_charging_order(waiting_car_infos.rename(columns={'waiting_car':'current_car'}))
# 保存充电信息和等待信息
charging_orders = pd.merge(charging_orders,station_info_table)
waiting_orders = pd.merge(waiting_orders,station_info_table)
# 计算充电时长
charging_orders['duration'] = (charging_orders['etime']-charging_orders['stime']).dt.total_seconds()
waiting_orders['duration'] = (waiting_orders['etime']-waiting_orders['stime']).dt.total_seconds()


In [6]:
charging_orders

,carid,order_id,stime,etime,station_id,duration,lon,lat,max_capacity,charge_speed_station
0,2,1,2023-07-13 22:30:00,2023-07-13 23:55:00,1253_ev_station,5100.0,121.588766,31.080315,31,7
1,91,25,2023-07-12 07:05:00,2023-07-12 11:55:00,1253_ev_station,17400.0,121.588766,31.080315,31,7
2,1718,519,2023-07-13 02:40:00,2023-07-13 05:15:00,1253_ev_station,9300.0,121.588766,31.080315,31,7
3,2856,876,2023-07-13 00:55:00,2023-07-13 07:15:00,1253_ev_station,22800.0,121.588766,31.080315,31,7
4,11410,3399,2023-07-12 13:05:00,2023-07-12 18:10:00,1253_ev_station,18300.0,121.588766,31.080315,31,7
...,...,...,...,...,...,...,...,...,...,...
72920,97445,29811,2023-07-12 07:35:00,2023-07-12 16:25:00,387_ev_station,31800.0,121.314348,31.195032,1,7
72921,97514,29828,2023-07-12 01:15:00,2023-07-12 07:35:00,961_ev_station,22800.0,121.487558,30.940100,1,7
72922,97730,29893,2023-07-12 15:40:00,2023-07-12 15:50:00,1376_ev_station,600.0,121.655917,31.190099,1,7
72923,98286,30066,2023-07-12 02:25:00,2023-07-12 07:55:00,512_ev_station,19800.0,121.353924,31.298023,1,7


In [7]:
taz = gpd.read_file(r"D:\北大深\华为充电桩网络优化\第二阶段城市场景\taz.geojson")

# 上海的栅格化参数
paramssh = {'slon': 120.88125,
        'slat': 30.7125,
        'deltalon': 0.0125,
        'deltalat': 0.008333,
        'theta': 0,
        'method': 'rect',
        'gridsize': 1000}
grid,paramssh = tbd.area_to_grid(taz,params=paramssh)

charging_order = charging_orders
charging_order['LONCOL'],charging_order['LATCOL'] = tbd.GPS_to_grid(charging_order['lon'],charging_order['lat'],params=paramssh)
charging_order_agg = charging_order.groupby(['LONCOL','LATCOL'])['carid'].count().reset_index()
charging_order_agg = pd.merge(grid[['LONCOL','LATCOL']],charging_order_agg,on=['LONCOL','LATCOL'],how="left")[['LONCOL','LATCOL','carid']]


In [8]:
charging_order_agg.fillna(0,inplace=True) 

In [9]:
charging_order_agg

,LONCOL,LATCOL,carid
0,95,-11,0.0
1,96,-11,0.0
2,94,-10,0.0
3,95,-10,0.0
4,96,-10,0.0
...,...,...,...
6617,31,138,0.0
6618,32,138,0.0
6619,33,138,0.0
6620,34,138,0.0


In [10]:
#潜在充电需求

In [11]:
car_infos = pd.read_csv('D:\北大深\华为充电桩网络优化\第二阶段城市场景\仿真结果\output_10W_v2\output_10W_v2\car_infos.csv')
step_length = 300

# 充电订单识别
car_infos['time'] = pd.to_datetime(car_infos['time'])
stay_infos = car_infos.sort_values(by=['carid','time'])
stay_infos['timegap'] = (-stay_infos['time'].diff(-1).dt.total_seconds()).fillna(1000000).astype(int)
stay_infos['etime'] = stay_infos['time'].shift(-1)
stay_order = stay_infos[stay_infos['timegap']>step_length][['carid','time','etime','soc','lon','lat']]
stay_order.columns = ['carid','stime','etime','soc','lon','lat']
stay_order['duration'] = (stay_order['etime']-stay_order['stime']).dt.total_seconds()
stay_order = stay_order[stay_order['duration']>0]
#聚合
stay_order=stay_order[(stay_order["soc"]<=50)&(stay_order["duration"]>=60*5)]  
stay_order['LONCOL'],stay_order['LATCOL'] = tbd.GPS_to_grid(stay_order['lon'],stay_order['lat'],params=paramssh)
stay_order_agg = stay_order.groupby(['LONCOL','LATCOL']).size().reset_index()
stay_order_agg = pd.merge(stay_order_agg,grid[['LONCOL','LATCOL']],on=['LONCOL','LATCOL'],how="right")
stay_order_agg.columns=['LONCOL','LATCOL',"pdemand"]
stay_order_agg.fillna(0,inplace=True)

In [12]:
stay_order_agg 

,LONCOL,LATCOL,pdemand
0,95,-11,0.0
1,96,-11,0.0
2,94,-10,0.0
3,95,-10,0.0
4,96,-10,0.0
...,...,...,...
6617,31,138,0.0
6618,32,138,0.0
6619,33,138,0.0
6620,34,138,0.0


In [13]:
#停车场 加油站 用地

In [14]:
poi=pd.read_excel(r"D:\北大深\充电桩环境影响\poi\gd_310000_poi.xlsx")
poi=poi[poi["pname"]=="上海市"]
poi['ttype'] = poi['type'].apply(lambda x: x.split(';')[0])
poi["lon"]=poi['location'].apply(lambda x: x.split(',')[0])
poi["lat"]=poi['location'].apply(lambda x: x.split(',')[1])

In [15]:
poi=poi[poi["type"].str.contains('停车场|加油站')]
#聚合
poi["lon"]=poi["lon"].astype("float")
poi["lat"]=poi["lat"].astype("float")
poi['LONCOL'],poi['LATCOL'] = tbd.GPS_to_grid(poi['lon'],poi['lat'],params=paramssh)
poi_agg = poi.groupby(['LONCOL','LATCOL']).size().reset_index()
poi_agg = pd.merge(poi_agg,grid[['LONCOL','LATCOL']],on=['LONCOL','LATCOL'],how="right")
poi_agg.columns=['LONCOL','LATCOL',"park"]
poi_agg.fillna(0,inplace=True)

In [16]:
poi_agg

,LONCOL,LATCOL,park
0,95,-11,0.0
1,96,-11,0.0
2,94,-10,0.0
3,95,-10,0.0
4,96,-10,0.0
...,...,...,...
6617,31,138,0.0
6618,32,138,0.0
6619,33,138,0.0
6620,34,138,0.0


In [17]:
#充电需求满足度

In [18]:
start_time=charging_orders["stime"].min()
end_time=charging_orders["stime"].max()
duration=end_time-start_time
station_chargetime=charging_orders.groupby(["station_id","lon","lat","max_capacity"])["duration"].sum().reset_index()
station_chargetime["duration"]=pd.to_timedelta(station_chargetime["duration"],unit='s')
station_chargetime["uti"]=station_chargetime["duration"]/(duration*station_chargetime["max_capacity"])
#聚合
station_chargetime['LONCOL'],station_chargetime['LATCOL'] = tbd.GPS_to_grid(station_chargetime['lon'],station_chargetime['lat'],params=paramssh)
station_chargetime_agg = station_chargetime.groupby(['LONCOL','LATCOL']).mean().reset_index()
station_chargetime_agg = pd.merge(station_chargetime_agg,grid[['LONCOL','LATCOL']],on=['LONCOL','LATCOL'],how="right")[['LONCOL','LATCOL','uti']]
station_chargetime_agg.columns=['LONCOL','LATCOL',"uti"]
station_chargetime_agg.fillna(0,inplace=True)

In [19]:
station_chargetime_agg

,LONCOL,LATCOL,uti
0,95,-11,0.0
1,96,-11,0.0
2,94,-10,0.0
3,95,-10,0.0
4,96,-10,0.0
...,...,...,...
6617,31,138,0.0
6618,32,138,0.0
6619,33,138,0.0
6620,34,138,0.0


In [20]:
#建站成本

In [21]:
price=pd.read_csv(r"D:\北大深\华为充电桩网络优化\第二阶段城市场景\POI\上海房价.csv")
price["geometry"]=gpd.points_from_xy(price["lon"],price["lat"])
price=gpd.GeoDataFrame(price,geometry=price["geometry"])
price.crs="EPSG:4326"
price=price.to_crs("EPSG:32651")
buffer=price.buffer(1000)
price=gpd.GeoDataFrame(price,geometry=buffer)

pricegrid=grid.to_crs("EPSG:32651")
pricegrid=gpd.sjoin(pricegrid,price)
pricegrid=pricegrid.groupby(["LONCOL","LATCOL"])["price"].mean().reset_index()

In [22]:
pricegrid["price"]=pricegrid["price"]*100*0.02*0.1*20+200000+400000+0.4*(pricegrid["price"]*100*0.02*0.1*20+200000+400000)
pricegrid=pd.merge(pricegrid,grid,how="right",on=['LONCOL','LATCOL'])
pricegrid.fillna(pricegrid["price"].min(),inplace=True)

In [23]:
pricegrid=pricegrid[['LONCOL','LATCOL',"price"]]

In [24]:
pricegrid

,LONCOL,LATCOL,price
0,95,-11,874440.0
1,96,-11,874440.0
2,94,-10,874440.0
3,95,-10,874440.0
4,96,-10,874440.0
...,...,...,...
6617,31,138,874440.0
6618,32,138,874440.0
6619,33,138,874440.0
6620,34,138,874440.0


In [25]:
del gridsum

NameError: name 'gridsum' is not defined

In [26]:
gridsum=pd.merge(charging_order_agg,stay_order_agg,on=['LONCOL','LATCOL'])
gridsum=pd.merge(gridsum,poi_agg,on=['LONCOL','LATCOL'])
gridsum=pd.merge(gridsum,station_chargetime_agg,on=['LONCOL','LATCOL'])
gridsum=pd.merge(gridsum,pricegrid,on=['LONCOL','LATCOL'])

In [27]:
gridsum

,LONCOL,LATCOL,carid,pdemand,park,uti,price
0,95,-11,0.0,0.0,0.0,0.0,874440.0
1,96,-11,0.0,0.0,0.0,0.0,874440.0
2,94,-10,0.0,0.0,0.0,0.0,874440.0
3,95,-10,0.0,0.0,0.0,0.0,874440.0
4,96,-10,0.0,0.0,0.0,0.0,874440.0
...,...,...,...,...,...,...,...
6617,31,138,0.0,0.0,0.0,0.0,874440.0
6618,32,138,0.0,0.0,0.0,0.0,874440.0
6619,33,138,0.0,0.0,0.0,0.0,874440.0
6620,34,138,0.0,0.0,0.0,0.0,874440.0


In [28]:
gridsum.columns=['LONCOL','LATCOL',"demand","pdemand","park","uti","price"]

In [29]:
gridsum.to_csv(r"D:\北大深\华为充电桩网络优化\选址\gridsum.csv",index=False)

In [31]:
grid.to_file(r"D:\北大深\华为充电桩网络优化\选址\grid.geojson")